In [1]:
from keras.models import Sequential
from keras.layers import Dense , MaxPool2D , Flatten 
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG19
from PIL import Image
import os
import numpy as np


In [2]:
labels = ['NoSpill','Spill']
img_size = 224
def get_data(data_dir):
    X = [] 
    Y = [] 
    for label in labels: 
        path = os.path.join(data_dir, label)
        class_num = labels.index(label)
        for img in os.listdir(path):
            try:
                image_path = os.path.join(path, img)
                img = Image.open(image_path)
                img = img.resize((img_size, img_size))
                resized_arr = np.array(img)
                X.append(resized_arr)
                Y.append(class_num)
            except Exception as e:
                print(e)
    return np.array(X), np.array(Y)

In [3]:
x_train, y_train = get_data(r"./dataset/train")
x_val, y_val = get_data(r"./dataset/Validation")
x_test, y_test =get_data(r"./dataset/test")

In [4]:
print("x_train:",x_train.shape)
print("y_train:",y_train.shape)
print("x_test:",x_test.shape)
print("y_test:",y_test.shape)
print("x_val:",x_val.shape)
print("y_val:",y_val.shape)

x_train: (2800, 224, 224, 3)
y_train: (2800,)
x_test: (600, 224, 224, 3)
y_test: (600,)
x_val: (600, 224, 224, 3)
y_val: (600,)


In [5]:

datagen = ImageDataGenerator(
       featurewise_center=False,  
      samplewise_center=False, 
       featurewise_std_normalization=False, 
       samplewise_std_normalization=False, 
       zca_whitening=False, 
      rotation_range = 30, 
      width_shift_range=0.1, 
    height_shift_range=0.1, 
       horizontal_flip = True, 
)
datagen.fit(x_train)
datagen.fit(x_val)
datagen.fit(x_test)

In [6]:
vgg19 = VGG19(weights='imagenet',include_top=False,input_shape=(224,224,3))

for layer in vgg19.layers[:19]:
    layer.trainable = False
    
model = Sequential()
model.add(vgg19)
model.add(MaxPool2D((2,2),strides=2))
model.add(Flatten())
model.add(Dense(2,activation='sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 7, 7, 512)         20024384  
                                                                 
 max_pooling2d (MaxPooling2  (None, 3, 3, 512)         0         
 D)                                                              
                                                                 
 flatten (Flatten)           (None, 4608)              0         
                                                                 
 dense (Dense)               (None, 2)                 9218      
                                                                 
Total params: 20033602 (76.42 MB)
Trainable params: 4728834 (18.04 MB)
Non-trainable params: 15304768 (58.38 MB)
_________________________________________________________________


In [7]:
opt = Adam(lr=0.001)
model.compile(optimizer=opt,loss="sparse_categorical_crossentropy",metrics ="accuracy")

In [8]:
from keras.callbacks import ReduceLROnPlateau
learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience = 2, verbose=1,factor=0.3, min_lr=0.000001)

In [11]:
history = model.fit(x_train,y_train,epochs =10  , validation_data = (x_val, y_val),shuffle=True,batch_size=32)

Epoch 1/10
88/88 [==============================] - 541s 6s/step - loss: 1.7212 - accuracy: 0.7721 - val_loss: 0.4554 - val_accuracy: 0.8533
Epoch 2/10
88/88 [==============================] - 510s 6s/step - loss: 0.6090 - accuracy: 0.8229 - val_loss: 0.5551 - val_accuracy: 0.8833
Epoch 3/10
88/88 [==============================] - 470s 5s/step - loss: 0.1498 - accuracy: 0.9521 - val_loss: 0.2495 - val_accuracy: 0.9333
Epoch 4/10
88/88 [==============================] - 469s 5s/step - loss: 0.0959 - accuracy: 0.9625 - val_loss: 0.1640 - val_accuracy: 0.9550
Epoch 5/10
88/88 [==============================] - 512s 6s/step - loss: 0.0295 - accuracy: 0.9857 - val_loss: 0.1017 - val_accuracy: 0.9800
Epoch 6/10
88/88 [==============================] - 520s 6s/step - loss: 0.0179 - accuracy: 0.9875 - val_loss: 0.0828 - val_accuracy: 0.9800
Epoch 7/10
88/88 [==============================] - 510s 6s/step - loss: 0.0104 - accuracy: 0.9929 - val_loss: 0.0892 - val_accuracy: 0.9883
Epoch 8/10
88

In [14]:
model.evaluate(x_test,y_test,batch_size=64)

10/10 [==============================] - 78s 8s/step - loss: 0.0490 - accuracy: 0.9933


[0.048989493399858475, 0.9933333396911621]

In [15]:
model.save("vgg_spill_detect_model.keras")